<a href="https://colab.research.google.com/github/yojulab/report_DebuggingGPT/blob/main/ModelBuildings/LLaMa_Alpaca_LoRA_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLaMa-Alpaca-LoRA 파인튜닝

### refer
##### youtube : https://youtu.be/aUXwVp4eUH4
##### colab : https://colab.research.google.com/drive/1B0kaiAPHt1n9SKDSmUE6asFmQVxfC3JS?usp=sharing#scrollTo=5Dw8-wbgIrJ0

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!nvidia-smi

Mon May 22 11:56:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download source code and install dependencies

- https://github.com/tloen/alpaca-lora.git
- 주의!: bitsandbytes가 현재 Linux에서만 동작함
- 주의!!: peft 버그 https://github.com/tloen/alpaca-lora/issues/293

In [3]:
!git clone -q https://github.com/kairess/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
#!pip uninstall transformers -y
#!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip uninstall peft -y
!pip install -q git+https://github.com/huggingface/peft.git

/content/alpaca-lora
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00


## Dataset

### Debugging 일지로 수집된 Instruction, Input, Response set.

- https://github.com/yojulab/report_DebuggingGPT/blob/main/Datas/self_instructs_output.json


```json
[
   {
        "instruction": "print(first)",
        "input": "----> 1 print(first)\n\nNameError: name 'first' is not defined",
        "output": "first = 'KIM'\nprint(first)\n"
    },
    {
        "instruction": "Empty1 = \" \", Empty2 = \" \", Empty3 = \" \"",
        "input": " Empty1 = \" \"  , Empty2 = \" \" , Empty3 = \" \"\n    ^\nSyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='?",
        "output": "Empty1 = \" \" \nEmpty2 = \" \" \nEmpty3 = \" \""
    },
     ...
]
```

In [81]:
!wget https://raw.githubusercontent.com/yojulab/report_DebuggingGPT/main/Datas/self_instructs_output.json -O self_instructs_output.json

--2023-05-22 13:56:00--  https://raw.githubusercontent.com/yojulab/report_DebuggingGPT/main/Datas/self_instructs_output.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12148 (12K) [text/plain]
Saving to: ‘self_instructs_output.json’

self_instructs_outp 100%[===================>]  11.86K  --.-KB/s    in 0s      

2023-05-22 13:56:00 (123 MB/s) - ‘self_instructs_output.json’ saved [12148/12148]



In [82]:
%ls

alpaca_data_cleaned_archive.json  generate.py
alpaca_data_gpt4.json             lengths.ipynb
alpaca_data.json                  LICENSE
data.json                         pyproject.toml
DATA_LICENSE                      README.md
docker-compose.yml                requirements.txt
Dockerfile                        result.jpg
export_hf_checkpoint.py           self_instructs_output.json
export_state_dict_checkpoint.py   templates/
finetune.py                       train.sh
flagged/                          utils/


In [83]:
import json

with open('self_instructs_output.json', 'r') as f:
    dataset = json.load(f)

dataset[5:10]

[{'instruction': 'E1 = "", E2 = ""',
  'input': 'line 1\n    E1 = "", E2 = ""\n    ^\nSyntaxError: invalid syntax. Maybe you meant \'==\' or \':=\' instead of \'=\'?',
  'output': 'E1 = ""\nE2 = ""'},
 {'instruction': 'Print("오상훈")',
  'input': '----> 1 Print("오상훈")\n\nNameError: name \'Print\' is not defined',
  'output': 'print("오상훈")'},
 {'instruction': 'Wind = "", Wish = ""',
  'input': 'line 1\n    Wind = "", Wish = ""\n    ^\nSyntaxError: invalid syntax. Maybe you meant \'==\' or \':=\' instead of \'=\'?',
  'output': 'Wind = ""\nWish = ""'},
 {'instruction': 'Print("한희수")',
  'input': '----> 1 Print("한희수")\n\nNameError: name \'Print\' is not defined',
  'output': 'print("한희수")'},
 {'instruction': 'if D = 9 :\n  B = 5\n  print(B)\nelse :\n  B = 200\n  print(B)',
  'input': "line 2\n    if D = 9 :\n       ^\nSyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='?",
  'output': 'if D == 9 :\n  B = 5\n  print(B)\nelse :\n  B = 200\n  print(B)'}]

## Finetune

기본 설정으로 했을 때:
- 3h/1epoch in NVIDIA A100
- 12h/1epoch in NVIDIA T4

### Create a custom template

Default Alpaca

```json
{
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

In [84]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

### Hyperparameters

https://github.com/tloen/alpaca-lora/blob/0e1a5d52a460d14aea2325e43c302972badb9cdd/finetune.py#L28

In [ ]:
# 권장 학습 파라메터
# !python finetune.py \
#     --base_model 'decapoda-research/llama-7b-hf' \
#     --data_path 'self_instructs_output.json' \
#     --output_dir './output' \
#     --num_epochs 3 \
#     --learning_rate 5e-5 \
#     --val_set_size 2000 \
#     --batch_size 512 \
#     --micro_batch_size 16 \
#     --prompt_template_name 'custom'

In [88]:
# 수정 학습 파라메터
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'self_instructs_output.json' \
    --output_dir './output' \
    --num_epochs 1 \
    --learning_rate 5e-4 \
    --val_set_size 10 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

2023-05-22 14:05:16.813852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/

In [ ]:
#!mkdir /content/drive/MyDrive/LLaMa-Alpaca-LoRA
!cp -a output_third /content/drive/MyDrive/LLaMa-Alpaca-LoRA

## Test on Gradio

--load_8bit

In [ ]:
# 권장 실행 명령어
# !python generate.py \
#     --base_model 'decapoda-research/llama-7b-hf' \
#     --lora_weights '/content/drive/MyDrive/LLaMa-Alpaca-LoRA/output' \
#     --prompt_template 'custom' \
#     --share_gradio

In [78]:
# 수정 실행 명령어
!python generate.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --lora_weights '/content/drive/MyDrive/LLaMa-Alpaca-LoRA/output_third' \
    --prompt_template 'custom' \
    --share_gradio

2023-05-22 13:14:01.756266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/